In [1]:
from transformers import pipeline

ModuleNotFoundError: No module named 'transformers'

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", framework="tf")

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
import pandas as pd
from streamz import Stream
import time
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor

df = pd.read_csv('./data/topical_chat.csv')

# Function to simulate streaming data from the CSV
def csv_data_stream(dataframe, interval=1):
    for _, row in dataframe.iterrows():
        yield row.to_dict()
        sleep_duration = random.uniform(0.1, interval)

        time.sleep(sleep_duration)


def count_data_points(data_batch):
    count = len(data_batch)
    return count

source = Stream()

data_batch = []

def process_pipeline(data, summarizer=summarizer):
    global data_batch
    data_batch.append(data)

    print(data['message'])
    
    # If the batch has reached the 1-minute mark (simulated by 5 data points here), summarize the data points
    if len(data_batch) >= 5:  # Adjust according to your interval
        text = " ".join([d['message'] for d in data_batch])
        summary = summarizer(text, max_length=55, min_length=40, do_sample=False)
        print(f"\nSummary : {summary[0]['summary_text']}\n")
        data_batch = []  # Clear the batch after counting

source.map(process_pipeline).sink(lambda x: x)

# Simulate streaming data and push it to the stream
for data in csv_data_stream(df):
    source.emit(data)


 Are you a fan of Google or Microsoft?
 Both are excellent technology they are helpful in many ways. For the security purpose both are super.
 I'm not  a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense. 
 Google provides online related services and products, which includes online ads, search engine and cloud computing.
 Yeah, their services are good. I'm just not a fan of intrusive they can be on our personal lives. 

Summary : I'm not a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense. I'm just not a fan of intrusive they can be on our personal lives.

 Google is leading the alphabet subsidiary and will continue to be the Umbrella company for Alphabet internet interest.
 Did you know Google had hundreds of live goats to cut the grass in the past? 
 It is very interesting. Google provide "Chrome OS" which is a light weight OS. Google provided a lot of hardware mainly in 2010 to 2015. 

Your max_length is set to 55, but your input_length is only 49. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


 Yes he even won a cha cha championship in 1958
 Yeah. Did you know ballet dancers use a lot of shoes?

Summary : Did you know Bruce Lee was a dancer?  Yes he even won a cha cha championship in 1958. Did you know ballet dancers use a lot of shoes?  I love it.  Bye  do you like dance?

 Yes they go through 4 pairs in a single week
 Yeah that is a lot of shoes and also a lot of money
 Yeah. Babies are also really good at dancing when they hear music
 Yes and they smile more when they hit the beat
 Yeah pretty cute. Did you know Tupac was a ballet dancer?

Summary : Babies are really good at dancing when they hear music. They smile more when they hit the beat. They go through 4 pairs of shoes in a single week. Did you know Tupac was a ballet dancer?

 Yes he was in the nutcracker as the mouse king
 Yeah. DO you like SHakespeare?
 I love his work. Did you know he popularized many terms that we use to this day?
 Yes terms like, good riddance, in my heart of heart and such
 Yes. He lives at 

In [14]:
import pandas as pd
from streamz import Stream
import time
import random
import spacy
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer, AutoModelForSequenceClassification, AutoTokenizer

# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1

# Load CSV data
df = pd.read_csv('../data/topical_chat.csv')

bart_model_name = "facebook/bart-large-cnn"
bart_tokenizer = BartTokenizer.from_pretrained(bart_model_name)
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)
summarizer = pipeline("summarization", model=bart_model, tokenizer=bart_tokenizer, device=device)

nlp = spacy.load("en_core_web_sm")

sa_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sa_tokenizer = AutoTokenizer.from_pretrained(sa_model_name)
sa_model = AutoModelForSequenceClassification.from_pretrained(sa_model_name)
sa = pipeline("sentiment-analysis", model=sa_model, tokenizer=sa_tokenizer, device=device)

def extract_aspects_and_adjectives(text):
    doc = nlp(text)
    aspects = []
    adjectives = []
    for chunk in doc.noun_chunks:
        if chunk.root.pos_ == "PRON":
            continue
        aspect = chunk.root.text
        adjective = [child.text for child in chunk.root.children if child.dep_ == "amod"]
        aspects.append(aspect)
        adjectives.append(adjective[0] if adjective else "")
    return aspects, adjectives

def analyze_sentiment(aspects):
    results = [sa(aspect)[0] for aspect in aspects]
    return results

def format_aspects_sentiments(aspects, sentiments, adjectives):
    formatted_results = []
    for aspect, sentiment, adjective in zip(aspects, sentiments, adjectives):
        formatted_results.append(f"Aspect: {aspect}, Sentiment: {sentiment['label']}, Adjective: {adjective}")
    return "\n".join(formatted_results)

# Function to simulate streaming data from the CSV
def csv_data_stream(dataframe, interval=1):
    for _, row in dataframe.iterrows():
        yield row.to_dict()
        sleep_duration = random.uniform(0.1, interval)
        time.sleep(sleep_duration)

source = Stream()
data_batch = []

def process_pipeline(data, summarizer=summarizer):
    global data_batch
    data_batch.append(data)

    # If the batch has reached the 1-minute mark (simulated by 5 data points here), summarize the data points
    if len(data_batch) >= 5:  # Adjust according to your interval
        text = " ".join([d['message'] for d in data_batch])

        summary = summarizer(text, max_length=55, min_length=40, do_sample=False)[0]['summary_text']
        
        aspects, adjectives = extract_aspects_and_adjectives(summary)
        sentiments = analyze_sentiment(aspects)

        aspect_sentiment_summary = format_aspects_sentiments(aspects, sentiments, adjectives)

        final_summary = f"Summary: {summary}\n\nAspect and Sentiment Analysis:\n{aspect_sentiment_summary}"
        print(final_summary)
        
        data_batch = []  # Clear the batch after processing

source.map(process_pipeline).sink(lambda x: x)

# Simulate streaming data and push it to the stream
for data in csv_data_stream(df):
    source.emit(data)


 Are you a fan of Google or Microsoft?  Both are excellent technology they are helpful in many ways. For the security purpose both are super.  I'm not  a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense.   Google provides online related services and products, which includes online ads, search engine and cloud computing.  Yeah, their services are good. I'm just not a fan of intrusive they can be on our personal lives. 
Summary: I'm not a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense. I'm just not a fan of intrusive they can be on our personal lives.

Aspect and Sentiment Analysis:
Aspect: fan, Sentiment: POSITIVE, Adjective: huge
Aspect: Google, Sentiment: POSITIVE, Adjective: 
Aspect: monopoly, Sentiment: NEGATIVE, Adjective: 
Aspect: sense, Sentiment: POSITIVE, Adjective: 
Aspect: fan, Sentiment: POSITIVE, Adjective: 
Aspect: lives, Sentiment: POSITIVE, Adjective: personal
 Google 

KeyboardInterrupt: 